In [1]:
pip install --upgrade awscli boto3
!pip install --upgrade s3fs
pip install joblib
pip install boto3

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from joblib import dump



#Załaduj dane
data = pd.read_csv('s3://titanicapi/train.csv')

#Wstępne przetwarzanie
data['Age'].fillna(data['Age'].median(), inplace=True)
data = pd.get_dummies(data, columns=['Sex', 'Embarked', 'Pclass'])

#Podział na cechy i etykiety
X = data.drop(['Survived', 'Name', 'Ticket', 'Cabin'], axis=1)
y = data['Survived']

#Podział na zestawy treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Normalizacja
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#Model
model = LogisticRegression()
model.fit(X_train, y_train)

#Ocena modelu
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))
#Save the model to a file
model_filename = 'trained_model.joblib'
dump(model, model_filename)
import boto3

def upload_file_to_s3(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified, file_name is used
    """
    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except Exception as e:
        print(f'Error uploading file: {e}')
        return False
    return True

#Call the function
upload_successful = upload_file_to_s3('trained_model.joblib', 'titanicapi')
if upload_successful:
    print("Model uploaded successfully!")
else:
    print("Failed to upload the model.")


SyntaxError: invalid syntax (3165592501.py, line 1)